In [1]:
import earthaccess

In [2]:
import xarray as xr

In [3]:
auth = earthaccess.login(persist=True)

In [9]:
results = earthaccess.search_data(
    short_name = 'PACE_OCI_L3M_LANDVI',
    temporal = ("2024-03-01", "2025-11-30"),
    granule_name="*.MO.*.4km.*"
)

In [10]:
len(results)

21

In [12]:
import numpy as np
time = [
    np.datetime64(r["umm"]["TemporalExtent"]["RangeDateTime"]["BeginningDateTime"][:10])
    for r in results
]

In [13]:
fileset = earthaccess.open(results)
dataset = xr.open_mfdataset(fileset, concat_dim="time", combine="nested")
dataset = dataset.assign_coords(time=("time", time))

# Put in your preferred "box" so the files aren't so large
ds=dataset.sel(lat=slice(7.049167, 3.856667), lon=slice(3.956111, 9.040000))

QUEUEING TASKS | :   0%|          | 0/21 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/21 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/21 [00:00<?, ?it/s]

In [14]:
import rioxarray as rxr

ndvi = ds["ndvi"]
ndvi = ndvi.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
ndvi = ndvi.rio.write_crs("EPSG:4326")
ndvi.rio.to_raster("NDVI_March2024_November2025_timeseries.tif")